### Problem Statement

    Identify products at risk of backorder before the event occurs so that business has time to react.

### What is a Backorder?
    Backorders are products that are temporarily out of stock, but a customer is permitted to place an order against future inventory. A backorder generally indicates that customer demand for a product or service exceeds a company’s capacity to supply it. Back orders are both good and bad. Strong demand can drive back orders, but so can suboptimal planning.

### Data

Data file contains the historical data for the 8 weeks prior to the week we are trying to predict. The data was taken as weekly snapshots at the start of each week. Columns are defined as follows:

    sku - Random ID for the product

    national_inv - Current inventory level for the part

    lead_time - Transit time for product (if available)

    in_transit_qty - Amount of product in transit from source

    forecast_3_month - Forecast sales for the next 3 months

    forecast_6_month - Forecast sales for the next 6 months

    forecast_9_month - Forecast sales for the next 9 months

    sales_1_month - Sales quantity for the prior 1 month time period

    sales_3_month - Sales quantity for the prior 3 month time period

    sales_6_month - Sales quantity for the prior 6 month time period

    sales_9_month - Sales quantity for the prior 9 month time period

    min_bank - Minimum recommend amount to stock

    potential_issue - Source issue for part identified

    pieces_past_due - Parts overdue from source

    perf_6_month_avg - Source performance for prior 6 month period

    perf_12_month_avg - Source performance for prior 12 month period

    local_bo_qty - Amount of stock orders overdue

    deck_risk - Part risk flag

    oe_constraint - Part risk flag

    ppap_risk - Part risk flag

    stop_auto_buy - Part risk flag

    rev_stop - Part risk flag

    went_on_backorder - Product actually went on backorder. This is the target value.

#### Identify Right Error Metrics

    Based on the businees, identify right error metrics.

### Data Pre-processing
#### Loading the required libraries

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc

import tensorflow as tf

from keras.regularizers import l2

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

from keras.utils import to_categorical
from keras import optimizers
from keras.callbacks import Callback
from keras import backend as K

import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

###### fix random seed for reproducibility

    Generally, Keras gets its source of randomness from the NumPy random number generator.
    
    In addition, TensorFlow has its own random number generator that must also be seeded by calling the set_random_seed() function immediately after the NumPy random number generator, as follows:

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

Get current working directory

In [ ]:
PATH = os.getcwd()

Change directory

In [ ]:
os.chdir(PATH)

#### Loading the data

In [ ]:
data = pd.read_csv("BackOrders.csv",header=0)

#### Understand the Data

See the number row and columns

In [ ]:
data.<>

Display the columns

In [ ]:
data.<>

Display the index

In [ ]:
data.index

See the top rows of the data

In [ ]:
data.<>

Shows a quick statistic summary of your data using describe.

    For object data (e.g. strings or timestamps), the result’s index will include count, unique, top, and freq. 

        The top is the most common value.

        The freq is the most common value’s frequency.

In [ ]:
data.<>

Display data type of each variable

In [ ]:
data.<>

#### Observations

    sku is Categorical but is interpreted as int64 

    potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder are categorical but is interpreted as object. 

#### Convert all the attributes to appropriate type

Data type conversion

    Using astype('category') to convert potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder attributes to categorical attributes.


In [ ]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].<>

Display data type of each variable

In [ ]:
data.dtypes

##### Delete sku attribute

In [ ]:
np.size(np.unique(data.sku))

In [ ]:
data.<>

#### Missing Data

    Missing value analysis and dropping the records with missing values

In [ ]:
data.isnull().sum()

Observing the number of records before and after missing value records removal

In [ ]:
print (data.shape)

Since the number of missing values is about 5%. For initial analysis we ignore all these records

In [ ]:
data = data.<>

In [ ]:
print(data.isnull().sum())
print("----------------------------------")
print(data.shape)

#### Converting Categorical to Numeric

For some of the models all the independent attribute should be of type numeric and ANN model is one among them.
But this data set has some categorial attributes.

'pandas.get_dummies' To convert convert categorical variable into dummy/indicator variables


In [ ]:
print (data.columns)

##### Creating dummy variables.

If we have k levels in a category, then we create k-1 dummy variables as the last one would be redundant. So we use the parameter drop_first in pd.get_dummies function that drops the first level in each of the category


In [ ]:
categorical_Attributes = data.select_dtypes(include=['category']).columns

In [ ]:
data = pd.get_dummies(columns=categorical_Attributes, data=data, prefix=categorical_Attributes, prefix_sep="_",drop_first=True)

In [ ]:
print (data.columns, data.shape)

#### Target attribute distribution

In [ ]:
pd.value_counts(data['went_on_backorder_Yes'])

In [ ]:
pd.value_counts(data['went_on_backorder_Yes'])/data['went_on_backorder_Yes'].count() * 100

#### Train-Test Split

Using sklearn.model_selection.train_test_split

    Split arrays or matrices into train and test subsets

In [ ]:
X, y = data.loc[:,data.columns!='went_on_backorder_Yes'].values, data.loc[:,'went_on_backorder_Yes'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify = data['went_on_backorder_Yes'])

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(pd.value_counts(y_train))
print(pd.value_counts(y_test))

In [ ]:
print(pd.value_counts(y_train)/y_train.size * 100)
print(pd.value_counts(y_test)/y_test.size * 100)

### Perceptron Model

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
perceptron_model = Sequential()

perceptron_model.<>

In [ ]:
perceptron_model.<>

In [ ]:
perceptron_model_history = perceptron_model.<>

##### Plot

In [ ]:
print(perceptron_model_history.history.keys())

In [ ]:
plt.plot(perceptron_model_history.history['acc'])
plt.plot(perceptron_model_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(perceptron_model_history.history['loss'])
plt.plot(perceptron_model_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

##### Predictions

In [ ]:
test_pred = perceptron_model.<>
train_pred = perceptron_model.<>

confusion_matrix_test = <>
confusion_matrix_train = <>

print(confusion_matrix_train)
print(confusion_matrix_test)

##### Train Test Accuracy, True Negative Rate and True Positive Rate

In [ ]:
Accuracy_Train=(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train= confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train= confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

print("-----------------------")

Accuracy_Test=(confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test= confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

### MLP with 2 layers

    1 hidden layer with 15 neurons

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
mlp_model1 = Sequential()

mlp_model1.<>
mlp_model1.<>

In [ ]:
mlp_model1.summary()

In [ ]:
mlp_model1.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model1_history = mlp_model1.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

##### Plot

In [ ]:
print(model1_history.history.keys())

In [ ]:
plt.plot(model1_history.history['acc'])
plt.plot(model1_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(model1_history.history['loss'])
plt.plot(model1_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_model1_pred = mlp_model1.predict_classes(X_train)
test_model1_pred = mlp_model1.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_model1_pred)
confusion_matrix_test = confusion_matrix(y_test, test_model1_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M1 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M1 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M1 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M1)
print("Train TPR: ",TPR_Train_M1)
print("Train Accuracy: ",Accuracy_Train_M1)

print("-----------------------")

Accuracy_Test_M1 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M1 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M1 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M1)
print("Test TPR: ",TPR_Test_M1)
print("Test Accuracy: ",Accuracy_Test_M1)

### MLP with 2 layers

    1 hidden layer with 20 neurons

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
mlp_model2 = Sequential()

mlp_model2.<>
mlp_model2.<>

In [ ]:
mlp_model2.summary()

In [ ]:
mlp_model2.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model2_history = mlp_model2.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

##### Plot

In [ ]:
print(model2_history.history.keys())

In [ ]:
plt.plot(model2_history.history['acc'])
plt.plot(model2_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(model2_history.history['loss'])
plt.plot(model2_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_model2_pred = mlp_model2.predict_classes(X_train)
test_model2_pred = mlp_model2.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_model2_pred)
confusion_matrix_test = confusion_matrix(y_test, test_model2_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M2 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M2 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M2 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M2)
print("Train TPR: ",TPR_Train_M2)
print("Train Accuracy: ",Accuracy_Train_M2)

print("-----------------------")

Accuracy_Test_M2 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M2 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M2 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M2)
print("Test TPR: ",TPR_Test_M2)
print("Test Accuracy: ",Accuracy_Test_M2)

### MLP with 2 layers

    1 hidden layer with 25 neurons

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
mlp_model3 = Sequential()

mlp_model3.<>
mlp_model3.<>

In [ ]:
mlp_model3.summary()

In [ ]:
mlp_model3.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model3_history = mlp_model3.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

##### Plot

In [ ]:
print(model3_history.history.keys())

In [ ]:
plt.plot(model3_history.history['acc'])
plt.plot(model3_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(model3_history.history['loss'])
plt.plot(model3_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_model3_pred = mlp_model3.predict_classes(X_train)
test_model3_pred = mlp_model3.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_model3_pred)
confusion_matrix_test = confusion_matrix(y_test, test_model3_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M3 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M3 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M3 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M3)
print("Train TPR: ",TPR_Train_M3)
print("Train Accuracy: ",Accuracy_Train_M3)

print("-----------------------")

Accuracy_Test_M3 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M3 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M3 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M3)
print("Test TPR: ",TPR_Test_M3)
print("Test Accuracy: ",Accuracy_Test_M3)

Observation:

    Based on the TPR, 2 layer MLP with 25 nodes hidden layer is best

### MLP with 3 layers

    1st hidden layer with 25 neurons
    2nd hidden layer with 15 neurons

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
mlp_model4 = Sequential()

mlp_model4.<>
mlp_model4.<>
mlp_model4.<>

In [ ]:
mlp_model4.summary()

In [ ]:
mlp_model4.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model4_history = mlp_model4.fit(X_train, y_train, epochs=150, batch_size=64, validation_split=0.2)

##### Plot

In [ ]:
print(model4_history.history.keys())

In [ ]:
plt.plot(model4_history.history['acc'])
plt.plot(model4_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(model4_history.history['loss'])
plt.plot(model4_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_model4_pred = mlp_model4.predict_classes(X_train)
test_model4_pred = mlp_model4.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_model4_pred)
confusion_matrix_test = confusion_matrix(y_test, test_model4_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M4 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M4 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M4 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M4)
print("Train TPR: ",TPR_Train_M4)
print("Train Accuracy: ",Accuracy_Train_M4)

print("-----------------------")

Accuracy_Test_M4 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M4 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M4 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M4)
print("Test TPR: ",TPR_Test_M4)
print("Test Accuracy: ",Accuracy_Test_M4)

### MLP with 3 layers

    1st hidden layer with 25 neurons
    2nd hidden layer with 20 neurons

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
mlp_model5 = Sequential()

mlp_model5.<>
mlp_model5.<>
mlp_model5.<>

In [ ]:
mlp_model5.summary()

In [ ]:
mlp_model5.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model5_history = mlp_model5.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

##### Plot

In [ ]:
print(model5_history.history.keys())

In [ ]:
plt.plot(model5_history.history['acc'])
plt.plot(model5_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(model5_history.history['loss'])
plt.plot(model5_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_model5_pred = mlp_model5.predict_classes(X_train)
test_model5_pred = mlp_model5.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_model5_pred)
confusion_matrix_test = confusion_matrix(y_test, test_model5_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M5 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M5 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M5 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M5)
print("Train TPR: ",TPR_Train_M5)
print("Train Accuracy: ",Accuracy_Train_M5)
print("-----------------------")

Accuracy_Test_M5 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M5 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M5 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M5)
print("Test TPR: ",TPR_Test_M5)
print("Test Accuracy: ",Accuracy_Test_M5)

Observation:

    Based on the TPR: 2 layer mlp, with 25 nodes in hiddent layer

### MLP with 4 layers

    1st hidden layer with 25 neurons
    2nd hidden layer with 20 neurons
    3nd hidden layer with 15 neurons

In [ ]:
np.random.seed(123)

tf.set_random_seed(123)

In [ ]:
mlp_model6 = Sequential()

mlp_model6.<>
mlp_model6.<>
mlp_model6.<>
mlp_model6.<>

In [ ]:
mlp_model6.summary()

In [ ]:
mlp_model6.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model6_history = mlp_model6.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

##### Plot

In [ ]:
print(model6_history.history.keys())

In [ ]:
plt.plot(model6_history.history['acc'])
# plt.plot(model4_history.history['val_acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(model6_history.history['loss'])
# plt.plot(model4_history.history['val_loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_model6_pred = mlp_model6.predict_classes(X_train)
test_model6_pred = mlp_model6.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_model6_pred)
confusion_matrix_test = confusion_matrix(y_test, test_model6_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)

print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

Observation:

    With respect to TPR, 2 layer mlp with 25 nodes in hidden layer is best 

## Find best parameter

#### Function for creation of model

In [ ]:
def model_def(h_activation, o_activation, kernel_init):
    

    
    return model 

#### Store the best parameters

In [ ]:
best_params={}

### Find the best Learning rate

    Using logarithmic scale randomly generate values between 0.0001 to 1 

In [ ]:
NUM_LR = 10 

lrs = []

lrs

In [ ]:
hist_loss = []

for lr in lrs:
    
    np.random.seed(123)
    tf.set_random_seed(123)
    
    lr_model = 
    
    # Compile model
    sgd = <>
    
    lr_model.<>
    
    # Fit the model
    lr_model_history = lr_model.<>
    
    
    hist_loss.append(lr_model_history.history['loss'])

In [ ]:
# Get the lr and loss Dataframe
loss_lr = pd.DataFrame([lrs, hist_loss]).T  

#Give the coloumn names
loss_lr.columns=['lr', 'loss']

#Sort the values and reset the index
loss_lr=<>
loss_lr

In [ ]:
#pick the top lr
best_params['best_learning_rate'] = <>

#### Build model with the best learning rate

#### Got the below value as best learning rate after different experiments

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)

In [ ]:
bst_lr_model = model_def(h_activation='tanh', o_activation='sigmoid', kernel_init='normal')

In [ ]:
bst_lr_model.summary()

In [ ]:
sgd = optimizers.SGD(lr=best_params['best_learning_rate'])
bst_lr_model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
bst_lr_model_history = bst_lr_model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, shuffle=True)

##### Plot

In [ ]:
print(bst_lr_model.history.history.keys())

In [ ]:
plt.plot(bst_lr_model.history.history['acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(bst_lr_model.history.history['loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_bst_lr_model_pred = bst_lr_model.predict_classes(X_train)
test_bst_lr_model_pred = bst_lr_model.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_bst_lr_model_pred)
confusion_matrix_test = confusion_matrix(y_test, test_bst_lr_model_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)
print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

#### Find the best Batch size 

In [ ]:
batch_sizes = [32, 64, 128, 256]

In [ ]:
history=[]

for batch_size in batch_sizes:
    
    np.random.seed(123)
    tf.set_random_seed(123)
    
    bs_model = model_def(h_activation='tanh', o_activation='sigmoid', kernel_init='normal')
    
     # Compile model
    sgd = <>
    bs_model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    #Fit the model
    bs_model_history = <>
    
    history.append(bs_model_history)

###### Summarize history for train loss

In [ ]:
for i in range(0, len(history)):
    plt.plot(history[i].history['loss'])
plt.title('Model Train Loss')
plt.ylabel('loss')
plt.xlabel('epochs')  
plt.legend(batch_sizes, loc='upper left')
plt.show()

###### Summarize history for test loss

In [ ]:
for i in range(0, len(history)):
    plt.plot(history[i].history['val_loss'])
plt.title('Model Test Loss')
plt.ylabel('loss')
plt.xlabel('epoch')  
plt.legend(batch_sizes, loc='upper left')
plt.show()

#### Build model with the best batch size

In [ ]:
best_params['best_batch_size'] = 256

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)
    
bst_bs_model = model_def(h_activation='tanh', o_activation='sigmoid', kernel_init='normal')

In [ ]:
sgd = optimizers.SGD(lr=best_params['best_learning_rate'])
bst_bs_model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
bst_bs_model_history = bst_bs_model.fit(X_train, y_train, epochs=100, 
                                        batch_size=best_params['best_batch_size'],
                                        validation_split=0.2, shuffle=True)

##### Plot

In [ ]:
print(bst_bs_model.history.history.keys())

In [ ]:
plt.plot(bst_bs_model.history.history['acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(bst_bs_model.history.history['loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_bst_bs_model_pred = bst_bs_model.predict_classes(X_train)
test_bst_bs_model_pred = bst_bs_model.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_bst_bs_model_pred)
confusion_matrix_test = confusion_matrix(y_test, test_bst_bs_model_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)

print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

#### Model with Relu as activation in hidden layer,Adam optimizer,Modified Xavier

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)

In [ ]:
relu_model= <>

In [ ]:
relu_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
relu_model_history = relu_model.fit(X_train, y_train, epochs=100,
                                    batch_size=best_params['best_batch_size'],
                                    validation_split=0.2, shuffle=True)

##### Plot

In [ ]:
print(relu_model.history.history.keys())

In [ ]:
plt.plot(relu_model.history.history['acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(relu_model.history.history['loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_relu_model_pred = relu_model.predict_classes(X_train)
test_relu_model_pred = relu_model.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_relu_model_pred)
confusion_matrix_test = confusion_matrix(y_test, test_relu_model_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)
print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

In [ ]:
best_params['best_h_activation']='relu'
best_params['best_kernel_init']='glorot_normal'
best_params['best_optimizer']='adam'

#### Model with Dropout

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)

In [ ]:
dropout_model = Sequential()
dropout_model.<>
dropout_model.<>
dropout_model.<>
dropout_model.<>
dropout_model.<>

In [ ]:
dropout_model.summary()

In [ ]:
dropout_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
dropout_model_history = dropout_model.fit(X_train, y_train, epochs=100,
                                          batch_size=best_params['best_batch_size'], 
                                          validation_split=0.2, shuffle=True)

##### Plot

In [ ]:
print(dropout_model.history.history.keys())

In [ ]:
plt.plot(dropout_model.history.history['acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(dropout_model.history.history['loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_dropout_model_pred = dropout_model.predict_classes(X_train)
test_dropout_model_pred = dropout_model.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_dropout_model_pred)
confusion_matrix_test = confusion_matrix(y_test, test_dropout_model_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)
print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

#### Model with Batch Normalization

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)

In [ ]:
bn_model = Sequential()
bn_model.<>
bn_model.<>
bn_model.<>
bn_model.<>
bn_model.<>

In [ ]:
bn_model.summary()

In [ ]:
bn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
bn_model_history = bn_model.fit(X_train, y_train, epochs=100,
                                batch_size=best_params['best_batch_size'], 
                                validation_split=0.2)

##### Plot

In [ ]:
print(bn_model.history.history.keys())

In [ ]:
plt.plot(bn_model.history.history['acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(bn_model.history.history['loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_bn_model_pred = bn_model.predict_classes(X_train)
test_bn_model_pred = bn_model.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_bn_model_pred)
confusion_matrix_test = confusion_matrix(y_test, test_bn_model_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)
print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

#### model with weight decay

#### Find the best Weight Decay value

In [ ]:
weight_decay_list = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

In [ ]:
def l2_model_func(l2_value):
    # create model
    l2_model = Sequential()
    
    l2_model.<>
    l2_model.add(Dropout(0.5))
    l2_model.<>
    l2_model.add(Dropout(0.5))
    l2_model.<>
    
    # Compile model
    l2_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return l2_model   

In [ ]:
hist_loss=[]

for weight_decay in weight_decay_list:
    
    np.random.seed(123)
    tf.set_random_seed(123)
    
    l2_model = <>
    
    #Fit the model
    l2_model_history = l2_model.fit(X_train, y_train, epochs=1,
                                    validation_split=0.2,steps_per_epoch=50,
                                    validation_steps=50)
    
    hist_loss.append(l2_model_history.history['loss'])

In [ ]:
# Get the weight decay and loss Dataframe
loss_weight_decay = pd.DataFrame([weight_decay_list, hist_loss]).T  

#Give the coloumn names
loss_weight_decay.columns=['weight_decay', 'loss']

#Sort the values and reset the index
loss_weight_decay=loss_weight_decay.sort_values('loss').reset_index().drop('index',axis=1)
loss_weight_decay

In [ ]:
#pick the top weight decay
best_params['best_l2_value'] = loss_weight_decay.loc[:,'weight_decay'][0]

In [ ]:
best_params['best_l2_value']

#### Build model with the best l2 value

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)

In [ ]:
bst_l2_model = Sequential()

bst_l2_model.add(Dense(25, input_dim=21, activation='relu', kernel_regularizer=l2(best_params['best_l2_value']),kernel_initializer="glorot_normal"))
bst_l2_model.add(Dropout(0.5))
bst_l2_model.add(Dense(15, activation='relu',kernel_regularizer=l2(best_params['best_l2_value']), kernel_initializer="glorot_normal"))
bst_l2_model.add(Dropout(0.5))
bst_l2_model.add(Dense(1, activation='sigmoid',kernel_regularizer=l2(best_params['best_l2_value']), kernel_initializer="glorot_normal"))

# Compile model
bst_l2_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
bst_l2_model.summary()

In [ ]:
bst_l2_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
bst_l2_model_history = bst_l2_model.fit(X_train, y_train, epochs=100,
                                        batch_size=best_params['best_batch_size'], 
                                        validation_split=0.2, shuffle=True)

##### Plot

In [ ]:
print(bst_l2_model.history.history.keys())

In [ ]:
plt.plot(bst_l2_model.history.history['acc'])
plt.title('Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(bst_l2_model.history.history['loss'])
plt.title('Loss Function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

#### Predictions

In [ ]:
train_bst_l2_model_pred = bst_l2_model.predict_classes(X_train)
test_bst_l2_model_pred = bst_l2_model.predict_classes(X_test)

#### Getting evaluation metrics and evaluating model performance

In [ ]:
confusion_matrix_train = confusion_matrix(y_train, train_bst_l2_model_pred)
confusion_matrix_test = confusion_matrix(y_test, test_bst_l2_model_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [ ]:
Accuracy_Train_M6 =(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train_M6 = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train_M6 = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train_M6)
print("Train TPR: ",TPR_Train_M6)
print("Train Accuracy: ",Accuracy_Train_M6)

print("-----------------------")

Accuracy_Test_M6 = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test_M6 = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test_M6 = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test_M6)
print("Test TPR: ",TPR_Test_M6)
print("Test Accuracy: ",Accuracy_Test_M6)

In [ ]:
best_params

### Reference Links:

https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

https://keras.io/